In [1]:
import os
import vtk, slicer
from slicer.ScriptedLoadableModule import *

tissueloss_folder = ""
tissueloss_matrix_folder = ""
image_folder = ""
image_matrix_folder = ""

In [2]:
def createEmptyVolume(imageSize, imageSpacing, nodeName):
    """
    Creates an empty volume with the specified dimensions and spacing.

    Parameters:
    - imageSize (tuple of int): Dimensions of the image volume (width, height, depth).
    - imageSpacing (tuple of float): Spacing between voxels in each dimension (x, y, z).
    - nodeName (str): Name for the volume node to be created.

    Returns:
    - vtkMRMLScalarVolumeNode: The created volume node with the empty volume.
    """
    
    voxelType = vtk.VTK_FLOAT  # Define the type of voxel to be used

    # Create an empty image volume
    imageData = vtk.vtkImageData()
    imageData.SetDimensions(imageSize)
    imageData.AllocateScalars(voxelType, 1)

    # Apply thresholding to ensure the volume is empty
    thresholder = vtk.vtkImageThreshold()
    thresholder.SetInputData(imageData)
    thresholder.SetInValue(0)
    thresholder.SetOutValue(0)
    thresholder.Update()

    # Create and configure the volume node
    volumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode", nodeName)
    volumeNode.SetSpacing(imageSpacing)
    volumeNode.SetAndObserveImageData(thresholder.GetOutput())
    volumeNode.CreateDefaultDisplayNodes()
    volumeNode.CreateDefaultStorageNode()

    return volumeNode

In [ ]:
# List of tags and their corresponding values
channels = [
    '488',
    '561',
    '638',
]

# Uniform expansion for tags and their values
tags_and_values = [
    ("_0_100", (18.0, 0.0)),
    ("_100_200", (36.0, 18.0)),
    ("_200_300", (54.0, 36.0)),
    ("_300_400", (72.0, 54.0)),
    ("_400_500", (90.0, 72.0)),
    ("_500_600", (108.0, 90.0)),
    ("_600_700", (126.0, 108.0)),
    ("_700_800", (144.0, 126.0)),
    ("_800_900", (162.0, 144.0)),
    ("_900_1000", (180.0, 162.0)),
]

for channel in channels:
    """
    Process each channel and its corresponding tags to perform volume segmentation and transformation.

    Iterates through predefined channels and tags, applies various image processing steps, and saves the results.
    """
    
    print(f"Starting channel: {channel}")
    
    # Set spacings and downsampling factor based on the channel
    if channel == '638':
        tissueloss_spacing = (1.0, 1.0, 1.0)
        image_spacing = (0.5, 0.5, 0.5)
        ds = '2'
    else:
        tissueloss_spacing = (1.0, 1.0, 1.0)
        image_spacing = (1.0, 1.0, 1.0)
        ds = '4'
    
    for tag, (outer_value, inner_value) in tags_and_values:
        print(f"Processing tag: {tag}, outer_value: {outer_value}, inner_value: {inner_value}")

        output_folder = "" + channel + '/' + tag + '/'

        # Load and process tissueloss volume
        files = [f for f in os.listdir(tissueloss_folder) if f.endswith('.nrrd')]
        for filename in files:
            print("Processing...", filename)
            
            # Load tissueloss volume and apply spacing
            tissueloss_path = tissueloss_folder + filename
            tissueloss_volume = slicer.util.loadVolume(tissueloss_path, returnNode=True)
            tissueloss_volume_node = tissueloss_volume[1]
            tissueloss_volume_node.SetSpacing(tissueloss_spacing[0], tissueloss_spacing[1], tissueloss_spacing[2])

            # Load tissueloss transformation matrix if available
            tissueloss_matrix_filename = filename.replace("lesionboundary.nrrd", f"lesionboundary_{channel}_matrix.h5")
            tissueloss_matrix_path = tissueloss_matrix_folder + channel + "/" + tissueloss_matrix_filename
            if os.path.exists(tissueloss_matrix_path):
                slicer.util.loadTransform(tissueloss_matrix_path)
                tissueloss_transform_matrix_node = slicer.util.getNode(tissueloss_matrix_filename.replace('.h5', ''))
                tissueloss_volume_node.SetAndObserveTransformNodeID(tissueloss_transform_matrix_node.GetID())
            else:
                print("...Tissueloss matrix file not found.")

            # Load image volume and apply spacing
            image_filename = filename.replace('lesionboundary.nrrd', f'{channel}_{ds}xds.nrrd')
            image_path = image_folder + channel + f"/{ds}xds" + "/" + image_filename
            print(image_path)
            image_volume = slicer.util.loadVolume(image_path, returnNode=True)
            image_volume_node = image_volume[1]
            image_volume_node.SetSpacing(image_spacing[0], image_spacing[1], image_spacing[2])

            # Load image transformation matrix if available
            image_matrix_filename = filename.replace('lesionboundary.nrrd', 'T.h5')
            image_matrix_path = image_matrix_folder + image_matrix_filename
            if os.path.exists(image_matrix_path):
                slicer.util.loadTransform(image_matrix_path)
                image_transform_matrix_node = slicer.util.getNode(image_matrix_filename.replace('.h5', ''))
                image_volume_node.SetAndObserveTransformNodeID(image_transform_matrix_node.GetID())
            else:
                print("...Image matrix file not found.")

            # Define ROI based on image volume bounds
            bounds = [0]*6
            image_volume_node.GetRASBounds(bounds)

            # Calculate new ROI center and radius
            newCenter = [(bounds[0] + bounds[1]) / 2, (bounds[2] + bounds[3]) / 2, (bounds[4] + bounds[5]) / 2]
            newRadius = [(bounds[1] - bounds[0]) / 2, (bounds[3] - bounds[2]) / 2, (bounds[5] - bounds[4]) / 2]

            # Create ROI node and set its parameters
            roiNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsROINode")
            roiNode.SetXYZ(newCenter)
            roiNode.SetRadiusXYZ(newRadius)

            # Apply cropping to tissueloss volume
            cropVolumeLogic = slicer.modules.cropvolume.logic()
            cropVolumeParameterNode = slicer.vtkMRMLCropVolumeParametersNode()
            cropVolumeParameterNode.SetROINodeID(roiNode.GetID())
            cropVolumeParameterNode.SetInputVolumeNodeID(tissueloss_volume_node.GetID())
            cropVolumeParameterNode.SetVoxelBased(True)
            cropVolumeLogic.Apply(cropVolumeParameterNode)
            croppedVolume = slicer.mrmlScene.GetNodeByID(cropVolumeParameterNode.GetOutputVolumeNodeID())

            # Segment outer and inner ranges using segment editor
            outersegmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
            outersegmentationNode.CreateDefaultDisplayNodes()  # Only needed for display
            outersegmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(croppedVolume)
            addedSegmentID = outersegmentationNode.GetSegmentation().AddEmptySegment("outer range")

            outersegmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
            outersegmentEditorWidget.setMRMLScene(slicer.mrmlScene)
            outersegmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
            outersegmentEditorWidget.setMRMLSegmentEditorNode(outersegmentEditorNode)
            outersegmentEditorWidget.setSegmentationNode(outersegmentationNode)
            outersegmentEditorWidget.setMasterVolumeNode(croppedVolume)

            outersegmentEditorWidget.setActiveEffectByName("Threshold")
            effect = outersegmentEditorWidget.activeEffect()
            effect.setParameter("MinimumThreshold", "1")
            effect.setParameter("MaximumThreshold", "1")
            effect.self().onApply()

            outersegmentEditorWidget.setActiveEffectByName("Margin")
            effect = outersegmentEditorWidget.activeEffect()
            effect.setParameter("MarginSizeMm", str(outer_value))
            effect.self().onApply()

            outersegmentEditorWidget.setActiveEffectByName("Mask volume")
            effect = outersegmentEditorWidget.activeEffect()
            effect.parameterSetNode().SetNodeReferenceID("Mask volume.InputVolume", image_volume_node.GetID())
            effect.self().onApply()
            outputVolumeID = effect.parameterSetNode().GetNodeReferenceID("Mask volume.OutputVolume")
            outputVolume = slicer.mrmlScene.GetNodeByID(outputVolumeID)

            # Apply transformation to the output volume
            outputVolume.SetAndObserveTransformNodeID(image_transform_matrix_node.GetID())

            # Repeat segmentation process for inner range
            innersegmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
            innersegmentationNode.CreateDefaultDisplayNodes()
            innersegmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(croppedVolume)
            addedSegmentID = innersegmentationNode.GetSegmentation().AddEmptySegment("inner range")

            innersegmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
            innersegmentEditorWidget.setMRMLScene(slicer.mrmlScene)
            innersegmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
            innersegmentEditorWidget.setMRMLSegmentEditorNode(innersegmentEditorNode)
            innersegmentEditorWidget.setSegmentationNode(innersegmentationNode)
            innersegmentEditorWidget.setMasterVolumeNode(croppedVolume)

            innersegmentEditorWidget.setActiveEffectByName("Threshold")
            effect = innersegmentEditorWidget.activeEffect()
            effect.setParameter("MinimumThreshold", "1")
            effect.setParameter("MaximumThreshold", "1")
            effect.self().onApply()

            innersegmentEditorWidget.setActiveEffectByName("Margin")
            effect = innersegmentEditorWidget.activeEffect()
            effect.setParameter("MarginSizeMm", str(inner_value))
            effect.self().onApply()

            innersegmentEditorWidget.setActiveEffectByName("Mask volume")
            effect = innersegmentEditorWidget.activeEffect()
            effect.setParameter("Operation", "FILL_INSIDE")
            effect.parameterSetNode().SetNodeReferenceID("Mask volume.InputVolume", outputVolume.GetID())
            effect.self().onApply()
            outputVolumeID = effect.parameterSetNode().GetNodeReferenceID("Mask volume.OutputVolume")
            outputVolume = slicer.mrmlScene.GetNodeByID(outputVolumeID)

            # Apply transformation to the final output volume
            outputVolume.SetAndObserveTransformNodeID(image_transform_matrix_node.GetID())

            # Save the final output volume
            output_filename = filename.replace('lesionboundary', channel + '_' + tag)
            output_path = output_folder + output_filename
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            slicer.util.saveNode(outputVolume, output_path)
            print("...Completed")
            # slicer.mrmlScene.Clear()
        print(f"{tag} Finished!")